## Webscrape More Updated Data From Guttmacher's State Profiles

In [ ]:
# SCRAPING THE CONTENTS FROM https://states.guttmacher.org/policies/

# DEPENDENCIES
import requests
import json
import pandas as pd
import numpy as np

In [ ]:
url = "https://states.guttmacher.org/policies/guttmacher.json"

In [ ]:
response = requests.get(url).json()
# print(json.dumps(response, indent=4, sort_keys=True))

### Pulling the data from Guttmacher's JSON source

In [ ]:
all_abortion_df = pd.json_normalize(response)

In [ ]:
all_abortion_df.head()
print(type(all_abortion_df))
all_abortion_df.info()

In [ ]:
abortion_2020_data = all_abortion_df[["state", 
                                      "state_slug", 
                                      "policies_currently_in_effect",
                                      "environment_type", 
                                      "residents_in_state_born_outside_us", 
                                      "income_level_below_fpl", 
                                      "abortions_obtained_in_2017",
                                      "abortions_per_1000_women_aged_15_44_in_2017",
                                      "clinics_provided_abortion_in_2017", 
                                      "counties_without_abortion_provider_in_2017",
                                      "residents_in_counties_without_an_abortion_provider_in_2017" ]]
# abortion_2020_data.head()

In [ ]:
# # Adjusting columns to display percentages as floats instead of whole numbers
# abortion_2020_data.loc[:,"residents_in_state_born_outside_us"] *= 0.01
# abortion_2020_data.loc[:,"income_level_below_fpl"] *= 0.01
# abortion_2020_data.loc[:,"residents_in_counties_without_an_abortion_provider_in_2017"] *= 0.01

In [ ]:
# Changing number of abortions obtained from object to integer

abortion_2020_data.loc[:,"abortions_obtained_in_2017"] = abortion_2020_data["abortions_obtained_in_2017"].astype(str)

comma = ","
abortion_2020_data.loc[:,"abortions_obtained_in_2017"] = [each.replace(comma, "") for each in abortion_2020_data["abortions_obtained_in_2017"]]
abortion_2020_data.loc[:,"abortions_obtained_in_2017"] = abortion_2020_data["abortions_obtained_in_2017"].astype(int)

In [ ]:
# abortion_2020_data.head()

### Cleaning Driving Distance Data

In [ ]:
# Field "aowdd_15_49_after_14_weeks" dropped for lack of data
driving_distance = all_abortion_df[["state",
                                    "aowdd_15_49_6_weeks", 
                                    "aowdd_15_49_12_weeks",
                                    "aowdd_15_49_14_weeks",
                                    "aowdd_15_49_20_weeks",
                                    "aowdd_15_49_22_weeks", 
                                    "aowdd_15_49_24_weeks",
                                    "aowdd_15_49_after_24_weeks"]]
# driving_distance.head()

In [ ]:
# Driving distance is shown as a string in the dataset, e.g. ""Average one-way driving distance for women aged 15-49 in Arizona to the nearest clinic that performs abortions up to 20 weeks = 11 miles"
# New columns are created below to pull the number of average miles from each field
driving_distance.loc[:,"week_6"] = driving_distance.aowdd_15_49_6_weeks.str.split("\s+").str[-2]
driving_distance.loc[:,"week_12"] = driving_distance.aowdd_15_49_12_weeks.str.split("\s+").str[-2]
driving_distance.loc[:,"week_14"] = driving_distance.aowdd_15_49_14_weeks.str.split("\s+").str[-2]
driving_distance.loc[:,"week_20"] = driving_distance.aowdd_15_49_20_weeks.str.split("\s+").str[-2]
driving_distance.loc[:,"week_22"] = driving_distance.aowdd_15_49_22_weeks.str.split("\s+").str[-2]
driving_distance.loc[:,"week_24"] = driving_distance.aowdd_15_49_24_weeks.str.split("\s+").str[-2]
driving_distance.loc[:,"week_24_plus"] = driving_distance.aowdd_15_49_after_24_weeks.str.split("\s+").str[-2]

# driving_distance.head()

In [ ]:
# driving_distance.info()

In [ ]:
# Shows the average one-way driving distance for a woman to obtain an abortion at the given week threshold
driving_distance = driving_distance[["state", "week_6", "week_12", "week_14", "week_20", "week_22", "week_24", "week_24_plus"]]
driving_distance["week_24_plus"] = driving_distance["week_24_plus"].str.replace("in", "0")
# driving_distance.info()

In [ ]:
driving_distance2 = driving_distance.astype({"week_6": float, "week_12": float, "week_14": float, "week_20": float,"week_22": float, "week_24": float, "week_24_plus": float})
# driving_distance2

In [ ]:
driving_distance3 = driving_distance2.melt(id_vars=["state"], var_name="week_threshold", value_name="avg_oneway_miles").sort_values(["state"])
# driving_distance3.head()

### Cleaning Race/Ethnicity Data

In [ ]:
wora_ethnicities = all_abortion_df[["state","wora_in_state_15_49_breakdown"]]
# wora_ethnicities.head()

In [ ]:
wora_ethnicities2 = wora_ethnicities.wora_in_state_15_49_breakdown.str.split(" ", expand = True)
wora_ethnicities2 = wora_ethnicities2[[1,4,7,10,13]]
# wora_ethnicities2.head()

In [ ]:
wora_ethnicities3 = pd.merge(wora_ethnicities, wora_ethnicities2, left_index=True, right_index=True)
wora_ethnicities3.rename(columns={"wora_in_state_15_49_breakdown":"original_string", 1:"Hispanic", 4:"White", 7:"Black", 10:"Asian", 13:"Other"}, inplace=True)
# wora_ethnicities3.info()

In [ ]:
char = "%"
wora_ethnicities3["Hispanic"] = ([each.replace(char, "") for each in wora_ethnicities3["Hispanic"]])
wora_ethnicities3["White"] = ([each.replace(char, "") for each in wora_ethnicities3["White"]])
wora_ethnicities3["Black"] = ([each.replace(char, "") for each in wora_ethnicities3["Black"]])
wora_ethnicities3["Asian"] = ([each.replace(char, "") for each in wora_ethnicities3["Asian"]])
wora_ethnicities3["Other"] = ([each.replace(char, "") for each in wora_ethnicities3["Other"]])
# wora_ethnicities3.info()

In [ ]:
wora_ethnicities4 = wora_ethnicities3.astype({"Hispanic": float, "White": float, "Black": float, "Asian": float,"Other": float})

In [ ]:
wora_ethnicities4 = wora_ethnicities4[["Hispanic", "White", "Black", "Asian", "Other"]]

In [ ]:
wora_ethnicities5 = pd.merge(wora_ethnicities3, wora_ethnicities4, left_index = True, right_index = True)
wora_ethnicities5 = wora_ethnicities5[["state", "Hispanic_y", "White_y", "Black_y","Asian_y", "Other_y"]]
wora_ethnicities5 = wora_ethnicities5.rename(columns={"Hispanic_y":"Hispanic", "White_y":"White", "Black_y":"Black", "Asian_y":"Asian", "Other_y":"Other"})
# wora_ethnicities5.info()

In [ ]:
wora_ethnicities6 = wora_ethnicities5.melt(id_vars=["state"], var_name="Race/Ethnicity", value_name="Portion of WORA").sort_values(["state"])
wora_ethnicities6 = pd.DataFrame(wora_ethnicities6)
# wora_ethnicities6.head()

### Cleaning Age Breakdown

In [ ]:
ages = all_abortion_df[["state","wora_age_breakdown"]]
# print(ages)

In [ ]:
ages_2 = ages.wora_age_breakdown.str.split(" ", expand = True)
ages_2 = ages_2[[1,5,9,13,17, 21]]
# ages_2.head()

In [ ]:
ages_3 = pd.merge(ages, ages_2, left_index=True, right_index=True)
ages_3.rename(columns={"wora_age_breakdown":"original_string", 1:"15-17 Years", 5:"18-19 Years", 9:"20-24 Years", 13:"25-29 Years", 17:"30-39 Years", 21:"40-49 Years"}, inplace=True)
# ages_3.head()

In [ ]:
char = "%"
ages_3["15-17 Years"] = ([each.replace(char, "") for each in ages_3["15-17 Years"]])
ages_3["18-19 Years"] = ([each.replace(char, "") for each in ages_3["18-19 Years"]])
ages_3["20-24 Years"] = ([each.replace(char, "") for each in ages_3["20-24 Years"]])
ages_3["25-29 Years"] = ([each.replace(char, "") for each in ages_3["25-29 Years"]])
ages_3["30-39 Years"] = ([each.replace(char, "") for each in ages_3["30-39 Years"]])
ages_3["40-49 Years"] = ([each.replace(char, "") for each in ages_3["40-49 Years"]])


In [ ]:
ages_4 = ages_3.astype({"15-17 Years": float, 
                        "18-19 Years": float,
                        "20-24 Years": float,  
                        "25-29 Years": float, 
                        "30-39 Years": float,
                        "40-49 Years": float})

In [ ]:
ages_4 = ages_4[["15-17 Years", "18-19 Years","20-24 Years",  "25-29 Years","30-39 Years","40-49 Years"]]

In [ ]:
ages_5 = pd.merge(ages, ages_4, left_index = True, right_index = True)
ages_5 = ages_5[["state", "15-17 Years", "18-19 Years", "20-24 Years", "25-29 Years", "30-39 Years", "40-49 Years"]]

In [ ]:
ages_6 = ages_5.melt(id_vars=["state"], var_name="Age Group", value_name="Portion of WORA").sort_values(["state"])
# ages_6.head(15)
ages_6 = pd.DataFrame(ages_6)
ages_6.head()

### Cleaning the number of women of reproductive age (WORA) in the state from object to integer

In [ ]:
wora_in_state = all_abortion_df[["state", "wora_in_state_15_49"]]
# wora_in_state.head(20)

In [ ]:
# Changing the number of women of reproductive age from object to integer

# split "million" from the cell by splitting on space
appx_wora = wora_in_state.wora_in_state_15_49.str.split(expand = True)

# remove the non-digits from the first column in order to change the values to an integer
point = "."
appx_wora[0] = ([each.replace(comma, "") for each in appx_wora[0]])
appx_wora[0] = ([each.replace(point, "") for each in appx_wora[0]])
appx_wora[0] = appx_wora[0].astype(int)

# multiply the values that had been labeled "million" by 100,000 (removing the decimal earlier changed the scale of the number from a million to 100,000)
appx_wora.loc[appx_wora[1]=="million", "number"] = appx_wora[0]*100000
appx_wora["final_number"] = appx_wora["number"].fillna(appx_wora[0])

# merge this cleaned data back to the wora_in_state dataframe
wora_in_state_2 = pd.merge(wora_in_state, appx_wora["final_number"], left_index = True, right_index=True)


In [ ]:
# Combine dataframes wora_in_state_2, wora_ethnicities5, ages_5, driving_distance2 onto abortion_2020_data

abortion_data_1 = pd.merge(abortion_2020_data, wora_in_state_2, left_index=True, right_index=True)
abortion_data_2 = pd.merge(abortion_data_1,wora_ethnicities5, left_index=True, right_index=True)
abortion_data_3 = pd.merge(abortion_data_2,ages_5, left_index=True, right_index=True)
abortion_data = pd.merge(abortion_data_3,driving_distance2, left_index=True, right_index=True)

abortion_data.info()

In [ ]:
# NOTE: the field names from the webscrape were not updated to reflect the year of data they actually display
# Confirmed from Guttmacher data available for download here: 
    # https://www.poynter.org/reporting-editing/2022/the-us-saw-930160-abortions-in-2020-reversing-a-30-year-decline/
# It includes both 2017 and 2020 values for the number of abortions and rate per 1000 women
# Unless stated otherwise, assume data is from 2017

abortion_data = abortion_data.rename(columns={"state": "State", 
                                              "state_slug":"State URL Slug", 
                                              "environment_type":"Restrictive Category",
                                              "policies_currently_in_effect": "Current Policies",
                                              "residents_in_state_born_outside_us":"% WORA Residents Non-US", 
                                              "income_level_below_fpl": "% WORA Below FPL",
                                              "abortions_obtained_in_2017":"Abortions Obtained in 2020", 
                                              "abortions_per_1000_women_aged_15_44_in_2017": "Abortions per 1,000 WORA in 2020", 
                                              "clinics_provided_abortion_in_2017": "Abortion Clinics", 
                                              "counties_without_abortion_provider_in_2017":"Counties w/o Abortion Providers", 
                                              "residents_in_counties_without_an_abortion_provider_in_2017":"% WORA w/o County Abortion Provider", 
                                              "final_number": "Appx. Number of WORA", 
                                              "Hispanic":"% WORA: Hispanic", 
                                              "Black":"% WORA: Black", 
                                              "White":"% WORA: White", 
                                              "Asian":"% WORA: Asian", \
                                              "Other":"% WORA: Other", 
                                              "15-17 Years": "% WORA: 15-17 Years",
                                              "18-19 Years": "% WORA: 18-19 Years", 
                                              "20-24 Years": "% WORA: 20-24 Years", 
                                              "25-29 Years": "% WORA: 25-29 Years",
                                              "30-39 Years": "% WORA: 30-39 Years",
                                              "40-49 Years": "% WORA: 40-49 Years", 
                                              "week_6":"Avg. Distance (mi): 6 Weeks", 
                                              "week_12":"Avg. Distance (mi): 12 Weeks", 
                                              "week_14":"Avg. Distance (mi): 14 Weeks", 
                                              "week_20":"Avg. Distance (mi): 20 Weeks", 
                                              "week_22":"Avg. Distance (mi): 22 Weeks", 
                                              "week_24":"Avg. Distance (mi): 24 Weeks", 
                                              "week_24_plus":"Avg. Distance (mi): 24+ Weeks"})


In [ ]:
abortion_data2 = abortion_data[["State","State URL Slug","Restrictive Category","Current Policies","Abortions Obtained in 2020", 
                               "Abortions per 1,000 WORA in 2020","Appx. Number of WORA", "Abortion Clinics","Counties w/o Abortion Providers",
                               "% WORA w/o County Abortion Provider", "% WORA Residents Non-US","% WORA Below FPL",
                               "% WORA: Hispanic","% WORA: Black","% WORA: White","% WORA: Asian","% WORA: Other", 
                               "% WORA: 15-17 Years", "% WORA: 18-19 Years", "% WORA: 20-24 Years", 
                               "% WORA: 25-29 Years","% WORA: 30-39 Years", "% WORA: 40-49 Years",
                               "Avg. Distance (mi): 6 Weeks", "Avg. Distance (mi): 12 Weeks", "Avg. Distance (mi): 14 Weeks", 
                               "Avg. Distance (mi): 20 Weeks", "Avg. Distance (mi): 22 Weeks","Avg. Distance (mi): 24 Weeks","Avg. Distance (mi): 24+ Weeks"]]


In [ ]:
abortion_data2.info()

In [ ]:
abortion_data_cleaned = abortion_data2
abortion_data_cleaned

In [ ]:
ages_6.to_csv("WORA_ages_unpivoted.csv")
wora_ethnicities6.to_csv("WORA_ethnicity_unpivoted.csv")
driving_distance3.to_csv("Avg_Drive_Distance.csv")
abortion_data_cleaned.to_csv("Guttmacher_Webscrape.csv")
